In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

In [2]:
driver_options = Options()
driver_options.add_argument("accept-language=en-US")
driver_options.add_argument('headless');
# driver_options.add_argument('window-size=1200x600'); ##// optional

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s,options=driver_options)
driver.maximize_window()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\emman\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [3]:
INITIAL_WEBSITE = "https://www.jumia.sn/"
data = {}
# go to the root page and get header/menu with its sub-items

In [4]:
# Autres catégories -- None # issue to solve later

In [5]:
def process_article(article):
    """retrive needed information from an article"""
    try:
        name = article.find_element(by=By.CSS_SELECTOR, value='h3.name').text
        price = article.find_element(by=By.CSS_SELECTOR, value='div.prc').text
        img_link = article.find_element(by=By.CSS_SELECTOR, value='img.img').get_attribute('data-src')
        descrption_link = article.find_element(by=By.CSS_SELECTOR, value='a.core').get_attribute('href')
        try:
            old_price, decrease_percentage = article.find_element(by=By.CSS_SELECTOR, value='div.s-prc-w').text.split("\n")
        except:
            old_price, decrease_percentage = '', ''
        try:
            n_stars, n_votes = article.find_element(by=By.CSS_SELECTOR, value='div.rev').text.split("\n")
            # n_votes = int(n_votes.replace("(", "").replace(")", "")) # clean value and convert to int 
        except:
            n_stars, n_votes = '', ''
        try:
            article.find_element(by=By.CSS_SELECTOR, value='svg.ic.xprss')
            jumia_express = True
        except:
            jumia_express = False

        article_data = dict(name=name , price=price , descrption_link=descrption_link , img_link=img_link , jumia_express=jumia_express ,
                       old_price=old_price , decrease_percentage=decrease_percentage ,n_stars=n_stars, n_votes=n_votes  )
    except:
        article_data = None
    return article_data

In [6]:
def close_popup(driver):
    """close pop up frames"""
    try:
        [btn.click() for btn in driver.find_elements(by=By.CSS_SELECTOR, value='button.cls')]
    except:
        pass

In [7]:
driver.get(INITIAL_WEBSITE)
close_popup(driver)
menu_items = {}
menu_items_ = driver.find_elements(by=By.CSS_SELECTOR, value="div.flyout-w a.itm")
for cat in menu_items_: 
    link = cat.get_attribute('href')
    category_name = cat.find_element(by=By.CSS_SELECTOR, value='span').text
    menu_items[category_name] = link
    
print(f"Available categories : {len(menu_items)} \n" ) 

for (category_name, category_link) in menu_items.items():
    print(f"{category_name} -- {category_link} " )
    if category_link is not None :
        last_page_scraped = False
        next_page = category_link

        while not last_page_scraped:
            # go to a page to scrape articles
            driver.get(next_page)
            close_popup(driver)
            articles_ = driver.find_elements(by=By.CSS_SELECTOR, value='article.prd._fb.col.c-prd') # get article boxes
            articles = [ ]
            for article in articles_:
                article_data = process_article(article)
                if article_data:
                    articles.append(article_data)
            data[category_name] = articles 
            next_page_btn = driver.find_elements(by=By.CSS_SELECTOR, value='div.pg-w.-ptm.-pbxl a')[-2] 

            if next_page_btn.accessible_name not in ['Page suivante']:
                last_page_scraped = True
            else:
                next_page = next_page_btn.get_attribute("href")

            # break # uncomment if you just want to get first page articles

        n_results_str = driver.find_element(by=By.CSS_SELECTOR, value='p.-gy5.-phs').text
        print(f"Available articles in this category : {n_results_str.split()[0]} \n")
    
    # break # uncomment if you just want to get first category articles

Available categories : 12 

Téléphones & Tablettes -- https://www.jumia.sn/telephone-tablette/ 
Available articles in this category : 3490 

Supermarché -- https://www.jumia.sn/epiceries/ 
Available articles in this category : 2353 

Santé & Beauté -- https://www.jumia.sn/beaute-hygiene-sante/ 
Available articles in this category : 13417 

Électronique -- https://www.jumia.sn/electronique/ 
Available articles in this category : 2932 

Informatique -- https://www.jumia.sn/ordinateurs-accessoires-informatique/ 
Available articles in this category : 1430 

Electroménager -- https://www.jumia.sn/maison-bureau-electromenager/ 
Available articles in this category : 750 

Maison et bureau -- https://www.jumia.sn/maison-cuisine-jardin/ 
Available articles in this category : 10537 

Mode -- https://www.jumia.sn/fashion-mode/ 
Available articles in this category : 10140 

Produits pour bébés -- https://www.jumia.sn/bebe-puericulture/ 
Available articles in this category : 820 

Articles de sport

In [8]:
list_of_dfs = []
for category, articles in data.items():
    df = pd.DataFrame(articles)
    df.to_csv( os.path.join( "data", f"{category}.csv"), index=False  )
    
    df['main_category'] = category
    list_of_dfs.append(df)
    

global_df = pd.concat( list_of_dfs, axis=0)
global_df.to_csv( os.path.join( "data", f"GLOBAL.csv"), index=False  )

In [9]:
driver.close()